In [ ]:
!pip install simpletransformers

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [ ]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
model_dataset = input('Dataset used to train model: ')
training_type = input('Training type of model: ')
epochs = int(input('Number of epochs trained on model'))
model_name = f"{training_type}-{model_name}-{epochs}epochs"

testing_set = input('Dataset for prediction generation: ')

if training_type.lower() == 'regular':
    train_set = 'training'
    eval_set = 'testing'
    print('hi')
elif training_type.lower() == 'limited':
    train_set = 'testing'
    eval_set = 'training'
else:
    print('Please enter a valid training type')

In [ ]:
import pandas as pd
from simpletransformers.t5 import T5Model

for model_name in model_names:
    print(f'---- Generating Predictions for Model: {model_name}  on Dataset: {testing_set}----')

    model_args = {
        "overwrite_output_dir": True,
        "max_seq_length": 196,
        "eval_batch_size": 32,
        "num_train_epochs": 1,
        "use_multiprocessing": False,
        "num_beams": None,
        "do_sample": True,
        "max_length": 50,
        "top_k": 50,
        "top_p": 0.95,
        "num_return_sequences": 3,
    }

    model = T5Model("t5", model_name, args=model_args)
    df = pd.read_csv(f'emotion-labeled-data/{testing_set}/{testing_set}-{eval_set}_t5.tsv', sep="\t").astype(str)
    
    to_predict = [
        prefix + ": " + str(input_text)
        for prefix, input_text in zip(df["prefix"].tolist(), df["input_text"].tolist())
    ]
    
    preds = model.predict(to_predict)
    preds = [pred[0] for pred in preds]
    df["predictions"] = preds
    df = df.drop(['Unnamed: 0'], axis=1)
    df.to_csv(f'predictions-data/{model_name}/{testing_set}-predictions.tsv', sep="\t")